In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import  numpy as np
import cv2

In [2]:
from keras.preprocessing.image import array_to_img, ImageDataGenerator, img_to_array, load_img

In [3]:
GenerateImage= ImageDataGenerator(rotation_range=40, width_shift_range= .2, height_shift_range=.2, shear_range=.2,
                                  zoom_range=.2,horizontal_flip=True, fill_mode='nearest' )

In [21]:
img= load_img(r"C:\Users\Sakshi\Downloads\WhatsApp Image 2023-11-28 at 23.11.39 (1).jpeg")
x=img_to_array(img)
# to generate the image we need to load the img

In [22]:
x=x.reshape((1,)+ x.shape)
x.shape

(1, 1280, 720, 3)

In [23]:
i= 0
for batch in GenerateImage.flow(x, batch_size=1, save_to_dir="d:\preveiw", save_prefix='dd', save_format='JPG'):
    
    i=i+1
    if i>100:
        break

#  model:

In [25]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D,Flatten, Dense,Dropout,AvgPool2D

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Lambda, Flatten, Input
from keras.preprocessing  import image
from keras.models import Sequential
import glob
import keras

In [26]:
image_size=200

path1=r"D:\face_recognition" # path of folder
cate =["suraj","sakshi"]# picture containing folder
input_image=[]
for i in cate:
    folders=os.path.join(path1,i)
    label= cate.index(i)
    for image in os.listdir(folders):
        image_path= os.path.join(folders,image)
        image_array= cv2.imread(image_path)
        image_array=cv2.resize(image_array,(image_size,image_size))
        input_image.append([image_array,label])
        

In [27]:
len(input_image)

400

In [28]:
np.random.shuffle(input_image) # every time run the code shuffling is done

In [29]:
x=[]
y=[]

for x_values,labels in input_image:
    x.append(x_values)
    y.append(labels)

In [30]:
x= np.array(x)
y=np.array(y)

In [31]:
y.shape

(400,)

In [32]:
x_train = x[:300]
y_train = y[:300]

x_test = x[300:400]
y_test = y[300:400]

In [36]:
x_train = x_train/255
x_test = x_test/255

In [37]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [40]:
model=Sequential()
model.add(Conv2D(filters=50,kernel_size=(5,5),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',input_shape=x_train.shape[1:]))
model.add(Dense(2,activation='softmax'))

In [41]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [42]:
model.fit(x_train,y_train, epochs=5,batch_size=50)

Epoch 1/5


6/6 [==============================] - 22s 2s/step - loss: 14.4342 - accuracy: 0.5233
Epoch 2/5
6/6 [==============================] - 13s 2s/step - loss: 1.1303 - accuracy: 0.7067
Epoch 3/5
6/6 [==============================] - 12s 2s/step - loss: 0.3388 - accuracy: 0.8400
Epoch 4/5
6/6 [==============================] - 12s 2s/step - loss: 0.1211 - accuracy: 0.9833
Epoch 5/5
6/6 [==============================] - 12s 2s/step - loss: 0.1241 - accuracy: 0.9600


In [43]:
from sklearn.metrics import confusion_matrix,classification_report

In [44]:
pred=model.predict(x_train)
pred_final= pred.argmax(axis=1)

10/10 [==============================] - 2s 178ms/step


In [45]:
print(classification_report(y_train,pred_final))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       152
           1       1.00      0.91      0.95       148

    accuracy                           0.96       300
   macro avg       0.96      0.96      0.96       300
weighted avg       0.96      0.96      0.96       300



In [46]:
pred_t=model.predict(x_test)
pred_final_t= pred_t.argmax(axis=1)

4/4 [==============================] - 1s 169ms/step


In [47]:
print(classification_report(y_test,pred_final_t))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        46
           1       1.00      0.81      0.90        54

    accuracy                           0.90       100
   macro avg       0.91      0.91      0.90       100
weighted avg       0.92      0.90      0.90       100



In [48]:
model.save("face_recognitio.h5")

#  working

In [49]:
face_cascade= cv2.CascadeClassifier(r"D:\open cv\HaarCascade\HaarCascade\haarcascade_frontalface_default.xml")
def face_detection(img):
    face_img=img.copy()
    
    face_reacts=face_cascade.detectMultiScale(face_img)# this face is just x,y,height and breath of rectangle
    
    for(x,y,w,h) in face_reacts:
        cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,0,0),3)# frawing rectangle on face
    return face_img

In [50]:
eye_cascade = cv2.CascadeClassifier(r"C:\Users\Sakshi\HaarCascade\HaarCascade\haarcascade_eye.xml")

def eye_detection(img):
    face_img = img.copy()
    eye_rects =  eye_cascade.detectMultiScale(face_img) 
    for (x,y,w,h) in eye_rects:
        cv2.rectangle(face_img, (x,y), (x+w, y+h), (255,100,10),5)
    return face_img

In [52]:
from keras.models import load_model
model = load_model('face_recognitio.h5')
results={0:'suraj',  1:'sakshi'}
GR_dict={0:(0,255,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0)
haarcascade = cv2.CascadeClassifier(r"D:\open cv\HaarCascade\HaarCascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 51ms/step
